In [1]:
#!pip install doubleml
#!pip install --upgrade scikit-learn

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression, Ridge, Lasso, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import patsy
import warnings
from sklearn.base import BaseEstimator, clone
import statsmodels.api as sm
from IPython.display import Markdown
import os
import wget
import seaborn as sns
warnings.simplefilter('ignore')
np.random.seed(1234)

In [3]:

file_path = '/Users/frank/Downloads/NEW7080.dta'

data = pd.read_stata(file_path)

rename_dict = {
    "v1": "AGE",
    "v2": "AGEQ",
    "v4": "EDUC",
    "v5": "ENOCENT",
    "v6": "ESOCENT",
    "v9": "LWKLYWGE",
    "v10": "MARRIED",
    "v11": "MIDATL",
    "v12": "MT",
    "v13": "NEWENG",
    "v16": "CENSUS",
    "v18": "QOB",
    "v19": "RACE",
    "v20": "SMSA",
    "v21": "SOATL",
    "v24": "WNOCENT",
    "v25": "WSOCENT",
    "v27": "YOB"
}

data.rename(columns=rename_dict, inplace=True)

data['seg_mitad'] = data['QOB'].apply(lambda x: 1 if x in [3, 4] else 0)

print(data.head(10))

data.columns

   AGE   AGEQ  v3  EDUC  ENOCENT  ESOCENT  v7         v8  LWKLYWGE  MARRIED  \
0   40  40.50   1    11        0        0  13   8.955383  5.023558        1   
1   41  41.00   1    12        0        0  14   8.993365  5.061540        1   
2   41  41.50   1    12        0        0  14   9.310141  5.378315        1   
3   46  46.25   1    12        0        0  14   9.110465  5.178639        1   
4   46  46.00   1    16        0        0  18  10.310601  6.378776        1   
5   47  47.00   1    12        0        0  14   8.929236  4.997411        0   
6   48  48.75   1    14        0        0  16   9.205277  5.273452        1   
7   41  41.75   2     9        0        0  12   8.993365  5.061540        1   
8   47  47.50   1    12        0        0  14  10.758956  6.827130        1   
9   44  44.75   2    17        0        0  20   9.767066  5.835241        1   

   ...  RACE  SMSA  SOATL   v22  v23  WNOCENT  WSOCENT  v26   YOB  seg_mitad  
0  ...     0     1      1  10.0    5        0      

Index(['AGE', 'AGEQ', 'v3', 'EDUC', 'ENOCENT', 'ESOCENT', 'v7', 'v8',
       'LWKLYWGE', 'MARRIED', 'MIDATL', 'MT', 'NEWENG', 'v14', 'v15', 'CENSUS',
       'v17', 'QOB', 'RACE', 'SMSA', 'SOATL', 'v22', 'v23', 'WNOCENT',
       'WSOCENT', 'v26', 'YOB', 'seg_mitad'],
      dtype='object')

In [4]:
y = data['LWKLYWGE'].values
D = data['seg_mitad'].values
X = data.drop(['v3', 'v7', 'v8', 'v14', 'v15', 'v17', 'v22',
               'v22', 'v26','AGEQ','LWKLYWGE','v23'], axis=1)
X.columns

Index(['AGE', 'EDUC', 'ENOCENT', 'ESOCENT', 'MARRIED', 'MIDATL', 'MT',
       'NEWENG', 'CENSUS', 'QOB', 'RACE', 'SMSA', 'SOATL', 'v23', 'WNOCENT',
       'WSOCENT', 'YOB', 'seg_mitad'],
      dtype='object')

In [5]:
from sklearn.base import TransformerMixin, BaseEstimator
from formulaic import Formula

class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        if self.array:
            return df.values
        return df

In [14]:

transformer = FormulaTransformer("0 + poly(AGE, degree=6, raw=True) + poly(EDUC, degree=8, raw=True) "
                                 "+ MARRIED + RACE")

transformer = FormulaTransformer("0 + poly(AGE, degree=6, raw=True) + poly(EDUC, degree=8, raw=True) "
                                 "+ MARRIED + RACE", array=True)
                                 

In [15]:
'''
transformer = FormulaTransformer("0 + poly(AGE, degree=5, raw=True) "
                                 "+ MARRIED + RACE")

transformer = FormulaTransformer("0 + poly(AGE, degree=5, raw=True) "
                                 "+ MARRIED + RACE", array=True)
                                 '''

'\ntransformer = FormulaTransformer("0 + poly(AGE, degree=5, raw=True) "\n                                 "+ MARRIED + RACE")\n\ntransformer = FormulaTransformer("0 + poly(AGE, degree=5, raw=True) "\n                                 "+ MARRIED + RACE", array=True)\n                                 '

In [18]:
W = StandardScaler().fit_transform(transformer.fit_transform(X))

from econml.iv.dml import OrthoIV
cv = KFold(n_splits=5, shuffle=True, random_state=123)
plriv = OrthoIV(model_y_xw=LassoCV(cv=cv),
                model_t_xw=LogisticRegressionCV(cv=cv),
                model_z_xw=LogisticRegressionCV(cv=cv),
                cv=3, discrete_treatment=True, discrete_instrument=True, random_state=123)

In [19]:
from doubleml import DoubleMLData
dml_data = DoubleMLData.from_arrays(W, y, D)
print(dml_data)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16']
Instrument variable(s): None
No. Observations: 1063634

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063634 entries, 0 to 1063633
Columns: 18 entries, X1 to d
dtypes: float64(18)
memory usage: 146.1 MB



In [20]:
import doubleml as dml

class RegWrapper(BaseEstimator):

    def __init__(self, clf):
        self.clf = clf

    def fit(self, X, y):
        self.clf_ = clone(self.clf).fit(X, y)
        return self

    def predict(self, X):
        return self.clf_.predict_proba(X)[:, 1]

dml_plr_obj = dml.DoubleMLPLR(dml_data,
                               LassoCV(cv=cv),
                               RegWrapper(LogisticRegressionCV(cv=cv)),
                               RegWrapper(LogisticRegressionCV(cv=cv)),
                               n_folds=3)
print(dml_plr_obj.fit())

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16']
Instrument variable(s): None
No. Observations: 1063634

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: LassoCV(cv=KFold(n_splits=5, random_state=123, shuffle=True))
Learner ml_m: RegWrapper(clf=LogisticRegressionCV(cv=KFold(n_splits=5, random_state=123, shuffle=True)))
Out-of-sample Performance:
Learner ml_l RMSE: [[0.65691867]]
Learner ml_m RMSE: [[0.49855448]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err         t     P>|t|     2.5 %    97.5 %
d -0.003162  0.001277 -2.